<a href="https://colab.research.google.com/github/ArkS0001/VQA--Visual-Question-Answering/blob/main/VQA_quantized_llava.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers requests Pillow torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
# Load the LLaVA model and processor
def load_llava_model(model_id):
    import torch
    from transformers import AutoProcessor, LlavaForConditionalGeneration
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    ).to("cuda:0")  # Assuming GPU 0 is available
    processor = AutoProcessor.from_pretrained(model_id)
    return model, processor

# Function to get image from path (URL or local file)
def get_image_from_path(image_path):
    from PIL import Image
    import requests
    if image_path.startswith("http"):
        response = requests.get(image_path, stream=True)
        image = Image.open(response.raw)
    else:
        image = Image.open(image_path)
    return image

# Function to ask a question about an image
def ask_question(model, processor, image_path, question):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": question},
                {"type": "image"},
            ],
        },
    ]
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    image = get_image_from_path(image_path)
    inputs = processor(
        images=image, text=prompt, return_tensors='pt'
    ).to("cuda:0", torch.float16)

    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    answer = processor.decode(output[0][2:], skip_special_tokens=True)
    return answer

# Example usage
model_id = "llava-hf/llava-1.5-13b-hf"
model, processor = load_llava_model(model_id)
image_path = "http://images.cocodataset.org/val2017/000000039769.jpg"
question = "What are these?"
answer = ask_question(model, processor, image_path, question)
print(answer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/77.2k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

model-00005-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 12300 has 14.74 GiB memory in use. Of the allocated memory 14.64 GiB is allocated by PyTorch, and 2.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [1]:
pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [6]:
# Note: This code requires a GPU with sufficient memory to load the model.
# Make sure you have CUDA installed and a GPU available.
# You need to install bitsandbytes for 8-bit quantization: pip install bitsandbytes
# Additionally, you can set environment variable to manage memory allocation better:
# export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# Load the LLaVA model and processor with 8-bit quantization
def load_llava_model(model_id):
    import torch
    from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id, quantization_config=quantization_config, torch_dtype="auto"
    )
    processor = AutoProcessor.from_pretrained(model_id)
    return model, processor

# Function to get image from path (URL or local file)
def get_image_from_path(image_path):
    from PIL import Image
    import requests
    if image_path.startswith("http"):
        response = requests.get(image_path, stream=True)
        image = Image.open(response.raw)
    else:
        image = Image.open(image_path)
    return image

# Function to ask a question about an image
def ask_question(model, processor, image_path, question):
    image = get_image_from_path(image_path)
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": question},
                {"type": "image", "image": image},
            ],
        },
    ]
    inputs = processor(conversation)

    # Get the device from the model
    device = next(model.parameters()).device
    for key in inputs:
        if isinstance(inputs[key], torch.Tensor):
            inputs[key] = inputs[key].to(device, torch.float16)

    output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    answer = processor.decode(output[0][len(inputs["input_ids"][0]):], skip_special_tokens=True)
    return answer

# Example usage
model_id = "llava-hf/llava-1.5-13b-hf"
model, processor = load_llava_model(model_id)
image_path = "http://images.cocodataset.org/val2017/000000039769.jpg"
question = "What are these?"
answer = ask_question(model, processor, image_path, question)
print(answer)
# If you still get OutOfMemoryError, try excluding the vision encoder from quantization:
# def load_llava_model(model_id):
#     import torch
#     from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
#     quantization_config = BitsAndBytesConfig(
#         load_in_8bit=True,
#         modules_to_not_convert=["vision_tower"]
#     )
#     model = LlavaForConditionalGeneration.from_pretrained(
#         model_id, quantization_config=quantization_config
#     ).to("cuda:0")
#     processor = AutoProcessor.from_pretrained(model_id)
#     return model, processor

# Or, switch to the 7B model:
# model_id = "llava-hf/llava-1.5-7b-hf"

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 136.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 130.12 MiB is free. Process 4096 has 14.61 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 42.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)